In [1]:
import logging
logging.basicConfig(level=logging.INFO)

In [2]:
from cicada.communicator import SocketCommunicator

def main(communicator):
    print(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

Hello from player 1!Hello from player 0!



In [3]:
logging.getLogger("cicada.communicator").setLevel(logging.INFO)

SocketCommunicator.run(world_size=2, fn=main)

logging.getLogger("cicada.communicator").setLevel(logging.WARNING) # Put things back the way they were

INFO:cicada.communicator.socket.connect:Comm world player 0 listening to tcp://127.0.0.1:65298 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 listening to tcp://127.0.0.1:65300 for connections.
INFO:cicada.communicator.socket.connect:Comm world player 1 direct connect with ['tcp://127.0.0.1:65298', 'tcp://127.0.0.1:65300'].
INFO:cicada.communicator.socket.connect:Comm world player 0 direct connect with ['tcp://127.0.0.1:65298', 'tcp://127.0.0.1:65300'].
INFO:cicada.communicator.socket.connect:Comm world player 0 tcp://127.0.0.1:65298 accepted connection from tcp://127.0.0.1:65304
INFO:cicada.communicator.socket.connect:Comm world player 1 tcp://127.0.0.1:65304 connected to player 0 tcp://127.0.0.1:65298
INFO:cicada.communicator.socket:Comm world player 0 communicator ready.
INFO:cicada.communicator.socket:Comm world player 1 communicator ready.


Hello from player 0!Hello from player 1!



INFO:cicada.communicator.socket:Comm world player 1 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 communicator freed.
INFO:cicada.communicator.socket:Comm world player 0 result: None
INFO:cicada.communicator.socket:Comm world player 1 result: None


In [4]:
from cicada.logging import Logger

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [5]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.warning(f"Warning message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

In [6]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.error(f"Error message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

ERROR:root:Error message from player 0!
ERROR:root:Error message from player 1!


In [7]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.critical(f"Critical message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

CRITICAL:root:Critical message from player 0!
CRITICAL:root:Critical message from player 1!


In [8]:
logging.getLogger().setLevel(logging.DEBUG) # Don't filter DEBUG messages

def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.debug(f"Debug message from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

DEBUG:root:Debug message from player 0!
DEBUG:root:Debug message from player 1!


In [9]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=1)

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 1!


In [10]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    log.info(f"Hello only from player {communicator.rank}!", src=[0, 2, 4])

SocketCommunicator.run(world_size=5, fn=main);

INFO:root:Hello only from player 0!
INFO:root:Hello only from player 2!
INFO:root:Hello only from player 4!


In [11]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator, sync=False)
    log.info(f"Hello from player {communicator.rank}!")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Hello from player 0!
INFO:root:Hello from player 1!


In [12]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that never arrives ...
        if communicator.rank == 1:
            payload = communicator.recv(src=0, tag=42)
        
    except Exception as e:
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:cicada.communicator.socket:Comm world player 1 failed: Timeout('Tag LOGSYNC from player 0 timed-out after 5s')


In [13]:
def main(communicator):
    log = Logger(logger=logging.getLogger(), communicator=communicator)
    
    try:
        log.info(f"Player {communicator.rank} starting ...")
        
        # Wait for a message that will neve arrives
        if communicator.rank == 0:
            payload = communicator.recv(src=1, tag=42)
        
    except Exception as e:
        log.sync = False # Disable coordinated logging
        log.error(f"Player {communicator.rank} exception: {e}")

SocketCommunicator.run(world_size=2, fn=main);

INFO:root:Player 0 starting ...
INFO:root:Player 1 starting ...
ERROR:root:Player 0 exception: Tag 42 from player 1 timed-out after 5s
